# Pre-Processing Pipeline Figure Code

In [ ]:
# import libraries
import numpy as np
import random
import csv
import sklearn # ONLY IMPORT IF YOU WANT FLIP BUG FROM GUMPY
import matplotlib.pyplot as plt

In [ ]:
# load more libraries
!pip install git+https://github.com/gumpy-bci/gumpy
!pip install pyEDFlib

In [ ]:
# import loaded libraries
import gumpy
import pyedflib

In [ ]:
# get data at each step in the pre-processing stage for one trial

subjID = "001"
fileID = "03"
freq=160
subLen = 80

# open file
f = pyedflib.EdfReader("/content/drive/MyDrive/DL Final/files/S"+ subjID + "/S" + subjID + "R" + fileID +".edf")
annotations = f.readAnnotations()
trialStartTimes = annotations[0] # get start time of each trial
trialTypes = annotations[2] # get trial type (right, left or rest) for each trial

# initalize variables
raw_signal = np.zeros((64, subLen))
notch_signal = np.zeros((64, subLen))
filt_signal = np.zeros((64, subLen))
clean_signal = np.zeros((64, subLen))



for iTrial in range(2): # first trial is rest, loop to second
  if trialTypes[iTrial] != 'T0': # confirm trial is not rest
    startInd = int(trialStartTimes[iTrial] * freq)
    for iChan in range(64):
        read_sig = f.readSignal(iChan)
        raw_signal[iChan, :] = read_sig[startInd:(startInd+(subLen))] # crop raw signal to relevant portion
        notch_signal[iChan, :] = gumpy.signal.notch(raw_signal[iChan, :], cutoff=60, fs=160) # notch filter
        filt_signal[iChan, :] = gumpy.signal.butter_bandpass(notch_signal[iChan, :], 2, 60, fs=160, order = 5) # bandpass filter
        clean_signal[iChan, :] = np.squeeze(gumpy.signal.artifact_removal(filt_signal[iChan, :].reshape((-1, 1)))[0]) # artifact removal


f.close() # close the file

In [ ]:
# Create pre-processing steps figure for first four channels
# flip bug makes this change for every run of the above cell

iChan = 0

fig, axs = plt.subplots(3,4)

for iChan in range(4):
  axs[0,iChan].plot(raw_signal[iChan,:])
  axs[0,iChan].set_title("Channel " + str(iChan+1))
  axs[1,iChan].plot(filt_signal[iChan,:])
  axs[2,iChan].plot(clean_signal[iChan,:])
  axs[2,iChan].set_xlabel("Time Step")
  for iSig in range(3):
    axs[iSig,iChan].set_yticklabels([])
    axs[iSig,iChan].set_xticklabels([])

axs[0,0].set_ylabel("Raw Signal")
axs[1,0].set_ylabel("Filtered Signal")
axs[2,0].set_ylabel("Cleaned Signal")


In [ ]:
# Check how many of the signals were flipped

flip = 0
for iChan in range(64):
  if clean_signal[iChan,0] < 0:
    flip += 1

print(flip/64)

# Training Loss/Accuracy curves

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# load training histories
hist_withDL = np.load('/content/drive/MyDrive/DL Final/FusionWithFlipTrue.npy', allow_pickle=True).item()
hist_noDL = np.load('/content/drive/MyDrive/DL Final/FusionNoFlipTrue.npy', allow_pickle=True).item()

In [ ]:
# Make train history figure

fig, axs = plt.subplots(2,1)

axs[0].plot(hist_withDL['loss'], color='red', label='Training - With Data Leak')
axs[0].plot(hist_withDL['val_loss'], color='red', linestyle='dashed', label='Validation - With Data Leak')
axs[0].plot(hist_noDL['loss'], color='green', label='Training - Without Data Leak')
axs[0].plot(hist_noDL['val_loss'], color='green', linestyle='dashed', label='Validation - Without Data Leak')
axs[0].legend()
axs[0].set_title("Loss")
axs[0].set_ylabel("Loss")


axs[1].plot(hist_withDL['accuracy'], color='red', label='Training - With Data Leak')
axs[1].plot(hist_withDL['val_accuracy'], color='red', linestyle='dashed', label='Validation - With Data Leak')
axs[1].plot(hist_noDL['accuracy'], color='green', label='Training - Without Data Leak')
axs[1].plot(hist_noDL['val_accuracy'], color='green', linestyle='dashed', label='Validation - Without Data Leak')
axs[1].legend()
axs[1].set_title("Accuracy")
axs[1].set_ylabel("Accuracy")
axs[1].set_xlabel("Epoch")


fig.set_size_inches(10, 6)
fig.suptitle("Real Data")

In [ ]:
hist_noDL['accuracy']

# Loss/Accuracy Plots for Fake Data

In [ ]:
# load training histories

hist_withDL = np.load('/content/drive/MyDrive/DL Final/fakedata_WithFlip.npy', allow_pickle=True).item()
hist_noDL = np.load('/content/drive/MyDrive/DL Final/fakedata_NoFlip.npy', allow_pickle=True).item()

In [ ]:
# Make train history (for fake data)

fig, axs = plt.subplots(2,1)

axs[0].plot(hist_withDL['loss'], color='red', label='Training - With Data Leak')
axs[0].plot(hist_withDL['val_loss'], color='red', linestyle='dashed', label='Validation - With Data Leak')
axs[0].plot(hist_noDL['loss'], color='green', label='Training - Without Data Leak')
axs[0].plot(hist_noDL['val_loss'], color='green', linestyle='dashed', label='Validation - Without Data Leak')
axs[0].legend()
axs[0].set_title("Loss")
axs[0].set_ylabel("Loss")


axs[1].plot(hist_withDL['accuracy'], color='red', label='Training - With Data Leak')
axs[1].plot(hist_withDL['val_accuracy'], color='red', linestyle='dashed', label='Validation - With Data Leak')
axs[1].plot(hist_noDL['accuracy'], color='green', label='Training - Without Data Leak')
axs[1].plot(hist_noDL['val_accuracy'], color='green', linestyle='dashed', label='Validation - Without Data Leak')
axs[1].legend()
axs[1].set_title("Accuracy")
axs[1].set_ylabel("Accuracy")
axs[1].set_xlabel("Epoch")


fig.set_size_inches(10, 6)

fig.suptitle("Fake Data")

In [ ]:
hist_withDL['val_accuracy']